In [ ]:

import os
import base64
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()

endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
subscription_key = os.getenv("AZURE_OPENAI_API_KEY")
deployment = "gpt-4o-mini"

search_endpoint = os.getenv("AZURE_SEARCH_ENDPOINT")
search_key = os.getenv("AZURE_SEARCH_API_KEY")
search_index = "movie-rag"

show_citations = True

# Initialize Azure OpenAI client with key-based authentication
client = AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=subscription_key,
    api_version="2025-01-01-preview",
)

# IMAGE_PATH = "YOUR_IMAGE_PATH"
# encoded_image = base64.b64encode(open(IMAGE_PATH, 'rb').read()).decode('ascii')

# Prepare the chat prompt
chat_prompt = [
    { "role" : "system", "content" : "사용자가 정보를 찾는 데 도움이 되는 AI 도우미입니다." },
    { "role" : "user", "content" : "재밌는 영화를 추천해줘!" }
]

# Include speech result if speech is enabled
messages = chat_prompt

# Generate the completion
completion = client.chat.completions.create(
    model=deployment,
    messages=messages,
    max_tokens=6553,
    temperature=0.7,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None,
    stream=False,
    extra_body={
      "data_sources": [{
          "type": "azure_search",
          "parameters": {
            "endpoint": f"{search_endpoint}",
            "index_name": "movie-rag",
            "semantic_configuration": "movie-rag-semantic-configuration",
            "query_type": "vector_semantic_hybrid",
            "fields_mapping": {},
            "in_scope": True,
            # "role_information": "사용자가 정보를 찾는 데 도움이 되는 AI 도우미입니다.",
            "filter": None,
            "strictness": 3,
            "top_n_documents": 5,
            "authentication": {
              "type": "api_key",
              "key": f"{search_key}"
            },
            "embedding_dependency": {
              "type": "deployment_name",
              "deployment_name": "text-embedding-ada-002"
            }
          }
        }]
    }
)

# print(completion.to_json())
print(completion.choices[0].message.content)

if show_citations :
    print("\nCitations : ")

    for c in range(len(completion.choices[0].message.context["citations"])) :

        citation_title = completion.choices[0].message.context["citations"][c]["title"]
        citation_content = completion.choices[0].message.context["citations"][c]["content"][:30]

        print(f"Doc no.{c+1}")
        print(" Title: " + citation_title + "\n Content: " + citation_content + "\n")
    

추천할 만한 재미있는 영화 목록은 다음과 같습니다:

1. **그 뜨거운 하루 (Geu tteugeoun haru)** - 2012년작으로, 특별한 이야기를 담고 있습니다 [doc1].
2. **향로 (Hyangno)** - 2013년에 개봉한 이 영화는 흥미로운 전개로 관객을 사로잡습니다 [doc1].
3. **안올디 뉴스 (Anoldi nyuseu)** - 2021년작으로, 현대 사회를 반영한 재미있는 요소가 가득합니다 [doc1].
4. **I Like You (Niga joa)** - 2015년에 개봉한 이 영화는 사랑과 유머를 담고 있습니다 [doc1].
5. **Until** - 2010년에 개봉한 이 영화는 감동적인 스토리라인으로 많은 사랑을 받았습니다 [doc5].

이 영화들은 다양한 장르와 스토리를 가지고 있어 재미있게 감상할 수 있을 것입니다.


In [1]:
# print(completion.choices[0].message.content)

# if show_citations :
#     print("\nCitations : ")

#     for c in range(len(completion.choices[0].message.context["citations"])) :

#         citation_title = completion.choices[0].message.context["citations"][c]["title"]
#         citation_content = completion.choices[0].message.context["citations"][c]["content"][:30]

#         print(f"Doc no.{c+1}")
#         print(" Title: " + citation_title + "\n Content: " + citation_content + "\n")